In [180]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [181]:
df=pd.read_csv("https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv")

In [182]:
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [183]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1462 entries, 0 to 1461
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   lead_source               1334 non-null   object 
 1   industry                  1328 non-null   object 
 2   number_of_courses_viewed  1462 non-null   int64  
 3   annual_income             1281 non-null   float64
 4   employment_status         1362 non-null   object 
 5   location                  1399 non-null   object 
 6   interaction_count         1462 non-null   int64  
 7   lead_score                1462 non-null   float64
 8   converted                 1462 non-null   int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 102.9+ KB


In [184]:
df.dtypes


lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [185]:
#1 question

In [186]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [187]:
#finding categorical and numerical cols

In [188]:
category_cols=df.select_dtypes(include=['object']).columns
numerical_cols=df.select_dtypes(include=['int64','float64']).columns

In [189]:
category_cols,numerical_cols

(Index(['lead_source', 'industry', 'employment_status', 'location'], dtype='object'),
 Index(['number_of_courses_viewed', 'annual_income', 'interaction_count',
        'lead_score', 'converted'],
       dtype='object'))

In [190]:
#replace missing values

In [191]:
df[category_cols]=df[category_cols].fillna('NA')
df[numerical_cols]=df[numerical_cols].fillna(0.0)

In [192]:
df.isnull().sum()

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

In [193]:
#What is the most frequent observation (mode) for the column industry?

In [194]:
industry_mode=df['industry'].mode()[0]
print(f"the most frequent observation(mode) for the 'industry ' columns:{industry_mode}")

the most frequent observation(mode) for the 'industry ' columns:retail


In [195]:
df['industry'].value_counts()

industry
retail           203
finance          200
other            198
healthcare       187
education        187
technology       179
manufacturing    174
NA               134
Name: count, dtype: int64

In [196]:
# ans : retail

In [197]:
# q2: Question 2
#Create the correlation matrix for the numerical features of your dataset. In a correlation matrix, you compute the correlation coefficient between every pair of features.

#what are the two features that have the biggest correlation?

#interaction_count and lead_score
#number_of_courses_viewed and lead_score
#number_of_courses_viewed and interaction_count
#annual_income and interaction_count"""

In [198]:
# identify numerical columns

In [199]:
numerical_cols=df.select_dtypes(include=['int64','float64']).columns
print("Numerical columns:",numerical_cols)

Numerical columns: Index(['number_of_courses_viewed', 'annual_income', 'interaction_count',
       'lead_score', 'converted'],
      dtype='object')


In [200]:
#create a corelation matrix

In [201]:
correlation_matrix=df[numerical_cols].corr()
print(correlation_matrix)

                          number_of_courses_viewed  annual_income  \
number_of_courses_viewed                  1.000000       0.009770   
annual_income                             0.009770       1.000000   
interaction_count                        -0.023565       0.027036   
lead_score                               -0.004879       0.015610   
converted                                 0.435914       0.053131   

                          interaction_count  lead_score  converted  
number_of_courses_viewed          -0.023565   -0.004879   0.435914  
annual_income                      0.027036    0.015610   0.053131  
interaction_count                  1.000000    0.009888   0.374573  
lead_score                         0.009888    1.000000   0.193673  
converted                          0.374573    0.193673   1.000000  


In [202]:
# extract correaltion for the specified pairs

In [203]:
# Extract correlations for the specified pairs
pairs = [
    ('interaction_count', 'lead_score'),
    ('number_of_courses_viewed', 'lead_score'),
    ('number_of_courses_viewed', 'interaction_count'),
    ('annual_income', 'interaction_count')
]

# Print correlation for each pair
for col1, col2 in pairs:
    corr = correlation_matrix.loc[col1, col2]
    print(f"Correlation between {col1} and {col2}: {corr:.3f}")

Correlation between interaction_count and lead_score: 0.010
Correlation between number_of_courses_viewed and lead_score: -0.005
Correlation between number_of_courses_viewed and interaction_count: -0.024
Correlation between annual_income and interaction_count: 0.027


In [204]:
# Q3 Split the data
#Split your data in train/val/test sets with 60%/20%/20% distribution.
#Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.
#Make sure that the target value y is not in your dataframe.
#Question 3
#Calculate the mutual information score between y and other categorical variables in the dataset. Use the training set only.
#Round the scores to 2 decimals using round(score, 2).
#Which of these variables has the biggest mutual information score?

In [205]:
#data splitting

In [206]:
X=df.drop(columns=['converted'])
Y=df['converted']

In [207]:
# First split: 80% (train+val) and 20% test
X_temp, X_test, y_temp, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [208]:
# Second split: 60% train (75% of 80%) and 20% validation (25% of 80%)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)

In [209]:
#Verify the splits
print(f"Train set: {len(X_train)} samples ({len(X_train)/len(df)*100:.1f}%)")
print(f"Validation set: {len(X_val)} samples ({len(X_val)/len(df)*100:.1f}%)")
print(f"Test set: {len(X_test)} samples ({len(X_test)/len(df)*100:.1f}%)")

Train set: 876 samples (59.9%)
Validation set: 293 samples (20.0%)
Test set: 293 samples (20.0%)


In [210]:
# mutual information

In [211]:
# Identify categorical columns from the options
categorical_vars = ['industry', 'location', 'lead_source', 'employment_status']

# Calculate mutual information scores for each categorical variable
mi_scores = {}
for col in categorical_vars:
    if col in X_train.columns:  # Ensure the column exists
        score = mutual_info_score(X_train[col], y_train)
        mi_scores[col] = round(score, 2)
    else:
        print(f"Column {col} not found in the dataset.")

# Print mutual information scores
for col, score in mi_scores.items():
    print(f"Mutual information score for {col}: {score}")

# Identify the variable with the highest score
max_var = max(mi_scores, key=mi_scores.get)
print(f"The variable with the biggest mutual information score is: {max_var} ({mi_scores[max_var]})")

Mutual information score for industry: 0.01
Mutual information score for location: 0.0
Mutual information score for lead_source: 0.04
Mutual information score for employment_status: 0.01
The variable with the biggest mutual information score is: lead_source (0.04)


In [212]:
#final answer : leadsource

In [213]:
# Q4:Now let's train a logistic regression.
#Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
#Fit the model on the training dataset.
#To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
#model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
#Calculate the accuracy on the validation dataset and round it to 2 decimal digits.
#What accuracy did you get?

In [214]:
# Apply One-Hot Encoding to Categorical Variables
# Identify categorical columns
categorical_cols = X.select_dtypes(include=['object']).columns

# Create a ColumnTransformer to apply one-hot encoding to categorical columns
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'), categorical_cols)
    ],
    remainder='passthrough'  # Keep numerical columns as is
)

# Fit and transform the training data
X_train_encoded = preprocessor.fit_transform(X_train)

# Transform validation and test data
X_val_encoded = preprocessor.transform(X_val)
X_test_encoded = preprocessor.transform(X_test)


In [215]:
#training logistic regression model

In [216]:
# Initialize the model
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)

# Fit the model on the training data
model.fit(X_train_encoded, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [217]:
#Calculate Accuracy on the Validation Dataset

In [218]:
# Predict on the validation set
y_val_pred = model.predict(X_val_encoded)

# Calculate accuracy
accuracy = accuracy_score(y_val, y_val_pred)
accuracy_rounded = round(accuracy, 2)

print(f"Validation accuracy: {accuracy_rounded}")

Validation accuracy: 0.7


In [219]:
#final answer :0.7

In [220]:
#Q5:Question 5
#Let's find the least useful feature using the feature elimination technique.
#Train a model using the same features and parameters as in Q4 (without rounding).
#Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
#For each feature, calculate the difference between the original accuracy and the accuracy without the feature.
#Which of following feature has the smallest difference?

#'industry'
#'employment_status'
#'lead_score'
#Note: The difference doesn't have to be positive.

In [221]:
# Create a ColumnTransformer for one-hot encoding
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'), categorical_cols)
    ],
    remainder='passthrough'  # Keep numerical columns as is
)

# Fit and transform the training data
X_train_encoded = preprocessor.fit_transform(X_train)
X_val_encoded = preprocessor.transform(X_val)

In [222]:
#Train the Original Model and Calculate Baseline Accuracy

In [223]:
# Initialize and train the model
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train_encoded, y_train)

# Predict and calculate baseline accuracy
y_val_pred = model.predict(X_val_encoded)
baseline_accuracy = accuracy_score(y_val, y_val_pred)
print(f"Baseline accuracy (all features): {baseline_accuracy:.4f}")

Baseline accuracy (all features): 0.6997


In [224]:
#feautre elimination

In [225]:
# Get feature names after one-hot encoding
feature_names = preprocessor.get_feature_names_out()

# Dictionary to store accuracy differences
accuracy_differences = {}

# Features to test
features_to_test = ['industry', 'employment_status', 'lead_score']

for feature in features_to_test:
    # Create a new dataset excluding the feature
    if feature in categorical_cols:
        # For categorical features, drop all one-hot encoded columns related to the feature
        columns_to_drop = [col for col in X_train.columns if col == feature]
        X_train_temp = X_train.drop(columns=columns_to_drop)
        X_val_temp = X_val.drop(columns=columns_to_drop)
        
        # Update categorical columns for the new preprocessor
        new_categorical_cols = [col for col in categorical_cols if col != feature]
        new_preprocessor = ColumnTransformer(
            transformers=[
                ('cat', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'), new_categorical_cols)
            ],
            remainder='passthrough'
        )
    else:
        # For numerical features, drop the single column
        X_train_temp = X_train.drop(columns=[feature])
        X_val_temp = X_val.drop(columns=[feature])
        new_preprocessor = preprocessor  # Use the same preprocessor (numerical feature doesn't affect encoding)

    # Transform the data
    X_train_temp_encoded = new_preprocessor.fit_transform(X_train_temp)
    X_val_temp_encoded = new_preprocessor.transform(X_val_temp)

    # Train the model
    model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    model.fit(X_train_temp_encoded, y_train)

    # Predict and calculate accuracy
    y_val_pred_temp = model.predict(X_val_temp_encoded)
    new_accuracy = accuracy_score(y_val, y_val_pred_temp)

    # Calculate accuracy difference
    difference = baseline_accuracy - new_accuracy
    accuracy_differences[feature] = difference
    print(f"Accuracy without {feature}: {new_accuracy:.4f}, Difference: {difference:.4f}")

Accuracy without industry: 0.6997, Difference: 0.0000
Accuracy without employment_status: 0.6962, Difference: 0.0034
Accuracy without lead_score: 0.7065, Difference: -0.0068


In [226]:
#Identify the Feature with the Smallest Difference

In [227]:
# Find the feature with the smallest absolute difference
abs_differences = {feature: abs(diff) for feature, diff in accuracy_differences.items()}
least_useful_feature = min(abs_differences, key=abs_differences.get)
print(f"The feature with the smallest difference is: {least_useful_feature} ({abs_differences[least_useful_feature]:.4f})")

The feature with the smallest difference is: industry (0.0000)


In [228]:
#q6:Question 6
#Now let's train a regularized logistic regression.
#Let's try the following values of the parameter C: [0.01, 0.1, 1, 10, 100].
#Train models using all the features as in Q4.
#Calculate the accuracy on the validation dataset and round it to 3 decimal digits.
#Which of these C leads to the best accuracy on the validation set
#Note: If there are multiple options, select the smallest C.

In [229]:
# Create a ColumnTransformer for one-hot encoding
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'), categorical_cols)
    ],
    remainder='passthrough'  # Keep numerical columns as is
)

# Fit and transform the training data
X_train_encoded = preprocessor.fit_transform(X_train)
X_val_encoded = preprocessor.transform(X_val)

In [230]:
# List of C values to test
C_values = [0.01, 0.1, 1, 10, 100]

# Dictionary to store accuracies
accuracies = {}

# Train and evaluate a model for each C
for C in C_values:
    # Initialize and train the model
    model = LogisticRegression(solver='liblinear', C=C, max_iter=1000, random_state=42)
    model.fit(X_train_encoded, y_train)

    # Predict and calculate accuracy
    y_val_pred = model.predict(X_val_encoded)
    accuracy = accuracy_score(y_val, y_val_pred)
    accuracies[C] = round(accuracy, 3)
    print(f"Accuracy with C={C}: {accuracies[C]:.3f}")

Accuracy with C=0.01: 0.700
Accuracy with C=0.1: 0.700
Accuracy with C=1: 0.700
Accuracy with C=10: 0.700
Accuracy with C=100: 0.700


In [231]:
# Find the best accuracy and corresponding C
best_accuracy = max(accuracies.values())
best_C = min([C for C, acc in accuracies.items() if acc == best_accuracy])
print(f"Best C: {best_C} with accuracy: {accuracies[best_C]:.3f}")

Best C: 0.01 with accuracy: 0.700
